In [ ]:
import numpy as np
from numpy import linspace, sqrt, sin, exp, convolve, abs, where

def rect(x,T):
    return where(abs(x)<=T/2, 1, 0)



In [ ]:
#Test rect
import plotly.graph_objects as go

fig = go.Figure()

y= np.ones(100)
x = np.arange(-50,50,1)
print(x.shape)
print(y.shape)
fig.add_trace(go.Scatter(x=x,y=y*rect(x-5,10)))

fig.show()

In [ ]:
#la fréquence evolue de omega_0-delta_omega/2 à omega_0+delta_omega/2
T = 0.2 #Le temps d'une chirp
delta_omega = 7000 #delta de frequence autour de omega_0
omega_0 = 3500 #fréquence de base
delta_f = 7000/(2*np.pi)
f_0 = 3500/(2*np.pi)
A=1 #amplitude du signal
P = 5 #nombre de chirp envoyé
Tr = T*1.2 #distance entre les chirp. Si Tr=T

t = np.linspace(0,1,10000)

e_c = lambda t : np.exp(1j*(-delta_omega/2 + delta_omega*t/(2*T))*t)

def e_s(t, P, Tr):
    out = 0
    for p in range(P):
        out += e_c(t-p*Tr)*np.exp(-1j*omega_0*p*Tr)*rect(t-p*Tr,T)
    return out

def window(t,p,Tr):
    out = 0
    for p in range(P):
        out += rect(t-p*Tr,T)
    return out

def e_s_without_win(t,P,Tr):
    out = 0
    out = e_c(t)*rect(t,P*T)
    return out

s_e = e_s(t,P,Tr)*np.exp(1j*omega_0*t)
win = window(t,P,Tr)
s_e_without_win = e_s_without_win(t,P,Tr)*np.exp(1j*omega_0*t)
#print(np.real(s_e))

fig = go.Figure()
fig.add_trace(go.Scatter(x=t,y=np.real(s_e)))
fig.add_trace(go.Scatter(x=t,y=win))
fig.show()



In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=t,y=np.real(s_e)))
fig.add_trace(go.Scatter(x=t,y=win))
fig.show()

from scipy import signal
se_fft = np.fft.fft(s_e)
f, t, Zxx =signal.stft(s_e,10000,nperseg=200,return_onesided=False)



fig = go.Figure()


#fig.add_trace(go.Heatmap(x0=0,dx=0.0482,y0=-40000,dy=8,z=np.abs(Zxx)[np.argsort(f)],zmin=0,zmax=0.25,zsmooth = 'best'))
fig.add_trace(go.Heatmap(x=t,y=f[np.argsort(f)],z=np.abs(Zxx)[np.argsort(f)],zmin=0,zmax=0.25,zsmooth = 'best',colorbar=dict(title="Amplitude")))

#print(f[index])
fig.update_layout(yaxis_range=[-1000,1000], xaxis_title="Time [s]", yaxis_title="Frequency [Hz]",title_text="STFT of a moving chair @2.4GHz")
fig.show()

# Signal reçu

In [ ]:
v = 1e4 #[m/s]
d = 1.5*3e6

beta = 2*v/3e8
alpha = 2*d/3e8

r = A*np.exp(1j*omega_0*t*(1-beta))*e_s(t-alpha,P,Tr)*np.exp(-1j*omega_0*alpha)

fig = go.Figure()
fig.add_trace(go.Scatter(x=t,y=np.real(r)))
fig.add_trace(go.Scatter(x=t,y=win))
fig.show()

fig = go.Figure()

#fig.add_trace(go.Heatmap(x0=0,dx=0.0482,y0=-40000,dy=8,z=np.abs(Zxx)[np.argsort(f)],zmin=0,zmax=0.25,zsmooth = 'best'))
fig.add_trace(go.Heatmap(x=t,y=f[np.argsort(f)],z=np.abs(Zxx)[np.argsort(f)],zmin=0,zmax=0.25,zsmooth = 'best',colorbar=dict(title="Amplitude")))

#print(f[index])
fig.update_layout(yaxis_range=[-1000,1000], xaxis_title="Time [s]", yaxis_title="Frequency [Hz]",title_text="STFT of a moving chair @2.4GHz")
fig.show()

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import PyQt5

data = np.load('group1_clean_1.npz')
f0 = data[data.files[2]][0]
B = data[data.files[2]][1]
Ns = int(data[data.files[2]][2])
Nr = int(data[data.files[2]][3])
Ts = data[data.files[2]][4]
Tr = data[data.files[2]][5]

c = 3*10**8
 
x = np.linspace(0, Ns*Nr*Ts , int(Ns*Nr))

# %matplotlib tk

d = np.linspace(0,10000,500)
v = np.linspace(-100,100,500).reshape((500,1))

print(d.shape)
print(v.shape)

bg = data[data.files[1]].reshape((Nr*Ns,1))
l = []
print(bg.shape)
X, Y = np.meshgrid(d, v,)
for i in range(10):
    frame = data[data.files[0]][i].reshape((Nr*Ns,1)) - bg
    print(frame.shape)
    C = np.zeros((d.shape[0],v.shape[0]),dtype='complex128')
    for n2 in range(0,int(Ns)):
        for n1 in range(0,int(Nr)):
            C += np.e**(1j * d*2/c * B * (n2*Ts/Tr) + 1j * n2 * Ts *2*np.pi*f0*2*v/c + 1j * n1 * Nr*Ts*2*np.pi*f0*2*v/c)*frame[n1*int(Ns) + n2]
    C = abs(C)**2
    l.append(C)

    # plt.clf()
    # plt.subplot(2,2,1)
    # plt.plot(x, abs(data[data.files[0]][i]))
 
    # plt.subplot(2,2,2)
    # plt.plot(x, abs(data[data.files[1]].reshape((1500, ))))
 
    # plt.subplot(2,2,2)
    # plt.plot(x, abs(data[data.files[0]][i]-data[data.files[1]].reshape((1500, ))))

# fig, ax = plt.subplots()
# xdata, ydata = [], []
# ln, = plt.plot([], [], 'ro')

# def init():
#     ax.set_xlim(0, 2*np.pi)
#     ax.set_ylim(-1, 1)
#     return ln,

# def update(frame):
#     ax.pcolormesh(X, Y, l[0]) 
#     ln.set_data(xdata, ydata)
#     return ln,

# ani = FuncAnimation(fig, update, frames=np.linspace(0, 99, 100,dtype='int'),
#                     init_func=init, blit=False)
# plt.show()

 
# plt.show()

In [ ]:
for i in range(70,80):
    fig, ax = plt.subplots(1, 1)
    ax.pcolormesh(X, Y, l[i])  
    plt.pause(Ns*Nr*Ts)